In [43]:
import xml.etree.cElementTree as et
import pandas as pd
import numpy as np
import os
import xlrd

DIR_PATH = 'nano-ds'
NANOISH_DIR = 'nanoish'
NONNANOISH_DIR = 'nonnano'

def getvalueofnode(node):
    return str(node.text) if node is not None else 'None'

def getEmptyIfNone(text):
    return text if text is not None else 'None'

In [44]:
def getMapDoiConceptFromExcel(rootDir):
    loc = (rootDir + '/' + 'Metadata.xlsx')
    wb = xlrd.open_workbook(loc)
    sheet = wb.sheet_by_index(1)
    dicts = {}
    for i in range(sheet.nrows):
        #value = sheet.cell_value(i, 4)
        #uValue = value.unicode()
        dicts[sheet.cell_value(i, 1)] = str(sheet.cell_value(i, 4))
    return dicts

def getXML2Text(rootDir):
    
    dfcols = ['type','doi','pipe','title','pipe','abstract','pipe','keywords','pipe','body','pipe','concepts']
    df = pd.DataFrame(columns=dfcols)
    typeDirs = [NANOISH_DIR,NONNANOISH_DIR]
    conceptDict = getMapDoiConceptFromExcel(DIR_PATH)
   
    for typeDir in typeDirs:
        for root, dirs, files in os.walk(rootDir + '/' + typeDir):
            for filename in files:
                if not filename.startswith('.'):
                    #print(rootDir + '/' + typeDir + '/' + filename)
                    parsed_xml = et.parse(rootDir + '/' + typeDir + '/' + filename)
                    for node in parsed_xml.getroot():
                        if node.tag == 'Journal':
                            doi = node.find('Volume/Issue/Article/ArticleInfo/ArticleDOI')
                            titleElem = node.find('Volume/Issue/Article/ArticleInfo')
                            abstractElem = node.find('Volume/Issue/Article/ArticleHeader')
                            body = node.find('Volume/Issue/Article/Body')
                            keywordGroup = node.find('Volume/Issue/Article/ArticleHeader')
                            ## Keyword sometimes have special tags as child nodes
                            
                            keyword_list = []
                            bodyPageList = []
                            abstractValue = ""
                            title = ""
                            
                            if titleElem != None:
                                if titleElem.find('ArticleTitle') != None:
                                    for articleTitle in titleElem.iter("ArticleTitle"):
                                        for tags in articleTitle.itertext():
                                            title += tags.strip()
                                else:
                                    title = 'None'
                            else:
                                title = 'None'
            
                            
                            if abstractElem != None:
                                if abstractElem.find('Abstract') != None:
                                    for para in abstractElem.iter("Para"):
                                        for abstract in para.itertext():
                                            abstractValue += abstract.strip()
                                else:
                                    abstractValue = 'None'
                            else:
                                abstractValue = 'None'
                        
 
                            if keywordGroup != None:        
                                if keywordGroup.find('keywordGroup') != None:
                                    for keyword in keywordGroup.iter("Keyword"):
                                        keywords = ""
                                        for txt in keyword.itertext():
                                            keywords += txt.strip()
                                        keyword_list.append(keywords)          
                                else:
                                    keyword_list.append('None')
                                keyword_str = ','.join(map(str,keyword_list))
                            else:
                                keyword_list.append('None')

                            if body != None:
                                if body.find('page') != None:
                                    pageList = []
                                    for page in body.iter("page"):
                                        pageList.append(page)
                                    pageWithoutTags = ""
                                    for pageTags in pageList[-1].itertext():
                                        pageWithoutTags += pageTags.strip()
                                    bodyPageList.append(pageWithoutTags)
                                    bodyPage = " ".join(map(str,bodyPageList))
                                else:
                                    paraList = []
                                    for para in body.iter("Para"):                                        
                                        paraList.append(para)
                                        paraWithoutTags = ""
                                    for paraTags in pageList[-1].itertext():
                                        paraWithoutTags += paraTags.strip()
                                    bodyPageList.append(paraWithoutTags)
                                    bodyPage = " ".join(map(str,bodyPageList))
                            else:
                                bodyPageList.append('None')
                                bodyPage = " ".join(map(str,bodyPageList))

                            df = df.append(pd.Series([str(typeDir),getvalueofnode(doi),'|',title,'|',abstractValue,'|',keyword_str, '|', bodyPage,'|', bytearray(getEmptyIfNone(conceptDict.get(getvalueofnode(doi))), 'utf-8').decode('utf-8')], index=dfcols), ignore_index=True)
                            
    df = df.sample(frac=1)
    #print(df)
    #df.to_csv("temp.csv",index=False)
    np.savetxt(r'dataV11_shuffle.txt', df.values, fmt='%s')
    
    
    

In [45]:
def main():
    #getMapDoiConceptFromExcel(DIR_PATH)
    getXML2Text(DIR_PATH)
    print('Done...')
main()

Done...
